<a href="https://colab.research.google.com/github/lee-j-j/LJJ/blob/main/ex10_mlp%2Ccnn_%EA%B0%9C%2C%EA%B3%A0%EC%96%91%EC%9D%B4_%EC%9D%B4%EC%A7%84%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 목표
- 개, 고양이 데이터 이미지를 배열로 변환 작업을 진행해보자 (이미지 데이터 전처리)
  - 이미지 자체를 저장소 업로드하는 것보다 배열로 변환했을 때 용량 소모가 적음
- mlp 이미지 데이터 이진 분류
- cnn 이미지 데이터 이진 분류
- 성능을 높이기 위한 노력을 진행해보자
  - 이미지 증식
  - 전이학습

In [4]:
# 개, 고양이 배열 데이터 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
data = np.load('./data/np_cat_vs_dog.npz')
data

In [7]:
len(data)
# 각각의 변수에 데이터 분리해서 담아주기
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

# 크기확인
print('훈련셋 : ', X_train.shape, y_train.shape)
print('테스트셋 : ', X_test.shape, y_test.shape)

훈련셋 :  (2000, 224, 224, 3) (2000,)
테스트셋 :  (1000, 224, 224, 3) (1000,)


#### mlp 모델 생성

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten # 1차원 펴주는 도구
from tensorflow.keras.callbacks import EarlyStopping # 조기 학습 중단 도구

In [12]:
# 1. 모델 설계
# 뼈대 구축
mlp_model = Sequential()

# 입력(3차원 → 1차원)
mlp_model.add(Flatten(input_shape = (224,224,3))) # 행,열,채널수(컬러) → 1차원 데이터 연결

# 중간층
mlp_model.add(Dense(units = 256, activation = 'relu'))
mlp_model.add(Dense(units = 128, activation = 'relu'))
mlp_model.add(Dense(units = 64, activation = 'relu'))

# 출력층 (이진분류 : units = 1, acti_f = sigmoid)
mlp_model.add(Dense(units = 1, activation = 'sigmoid'))
mlp_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 150528)            0         
                                                                 
 dense (Dense)               (None, 256)               38535424  
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 38576641 (147.16 MB)
Trainable params: 38576641 (147.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# 번역 : compile
# pc가 이해하도록 번역하는 작업
# loss, optimizer, metrics
mlp_model.compile(loss = 'binary_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])

In [15]:
# 조기학습중단 설정
# 검증 정확도 기준, 5번 기다림
f_ea = EarlyStopping(monitor = "val_accuracy", patience=5)

# 학습, 검증데이터 분리 30%, batchsize 64, 반복횟수 50, 조기학습중단 연결
mlp_his = mlp_model.fit(X_train, y_train, validation_split=0.3, epochs = 50, batch_size = 64, callbacks = [f_ea])
# validation_split 대신 validation_data를 쓸 때 : X_val, y_val 데이터를 따로 만들어 두어야 함!


Epoch 1/50
22/22 [==============================] - 1s 40ms/step - loss: 144.4199 - accuracy: 0.6414 - val_loss: 1260.6475 - val_accuracy: 0.0000e+00
Epoch 2/50
22/22 [==============================] - 1s 29ms/step - loss: 217.3420 - accuracy: 0.6436 - val_loss: 1199.0305 - val_accuracy: 0.0000e+00
Epoch 3/50
22/22 [==============================] - 1s 27ms/step - loss: 123.3529 - accuracy: 0.6800 - val_loss: 316.1369 - val_accuracy: 0.1500
Epoch 4/50
22/22 [==============================] - 1s 23ms/step - loss: 177.4574 - accuracy: 0.6429 - val_loss: 1332.3702 - val_accuracy: 0.0000e+00
Epoch 5/50
22/22 [==============================] - 0s 20ms/step - loss: 264.2211 - accuracy: 0.6286 - val_loss: 4.1427 - val_accuracy: 0.9567
Epoch 6/50
22/22 [==============================] - 0s 19ms/step - loss: 411.4988 - accuracy: 0.5957 - val_loss: 1161.0770 - val_accuracy: 0.0000e+00
Epoch 7/50
22/22 [==============================] - 0s 18ms/step - loss: 206.4472 - accuracy: 0.6350 - val_loss:

In [16]:
# test 데이터 활용해서 평가하기
# 모델.score(문제, 답) → 머신러닝 모델 평가하는 함수 (sklearn) * sk? science kit!
# 모델.evaluate(문제, 답) → 딥러닝 모델 평가하는 함수 (tensorflow.keras)
mlp_model.evaluate(X_test, y_test)

32/32 [==============================] - 0s 7ms/step - loss: 65.2584 - accuracy: 0.5720


[65.25836944580078, 0.5720000267028809]

#### cnn 모델 생성
- 1. 모델 설계 (cnn층)
- 2. 모델 학습 및 평가 방법 설정
- 3. 모델 학습 및 시각화
- 4. 모델 평가 및 예측

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout # Conv2D : 대상에 집중하게 만들어주는 역할

In [21]:
# cnn 신경망 구조를 설계
# 뼈대 생성
cnn_model = Sequential()

# 대상에 집중! → 중요한 특성에 집중! → 특성 추출
# 특성 추출부(층) - conv(합성곱)
cnn_model.add(Conv2D(32, (3,3), padding='same', input_shape=(224,224,3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size = 2))

cnn_model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
cnn_model.add(MaxPooling2D(pool_size = 2))

cnn_model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
cnn_model.add(MaxPooling2D(pool_size = 2))

# 개, 고양이 분류부(층) - mlp(입력(1차원으로 펴주는 기능 필요), 중간, 출력층(이진분류))
cnn_model.add(Flatten()) # 특성 추출부를 거친 후의 input_shape가 자동으로 전달 됨 - 생략
cnn_model.add(Dense(512, activation = 'relu')) # 중간층
cnn_model.add(Dense(1, activation = 'sigmoid')) # 출력층

# 모델 정보 확인
cnn_model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 112, 112, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 56, 56, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 28, 28, 128)      

In [22]:
# compile
cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit
cnn_his = cnn_model.fit(X_train, y_train, validation_split=0.3, epochs=50, batch_size=64, callbacks=[f_ea])

Epoch 1/50
22/22 [==============================] - 13s 207ms/step - loss: 301.5003 - accuracy: 0.5821 - val_loss: 0.9881 - val_accuracy: 0.0000e+00
Epoch 2/50
22/22 [==============================] - 2s 104ms/step - loss: 0.6347 - accuracy: 0.7143 - val_loss: 1.3514 - val_accuracy: 0.0000e+00
Epoch 3/50
22/22 [==============================] - 2s 99ms/step - loss: 0.5413 - accuracy: 0.7386 - val_loss: 1.3583 - val_accuracy: 0.0100
Epoch 4/50
22/22 [==============================] - 2s 100ms/step - loss: 0.4785 - accuracy: 0.7736 - val_loss: 1.8339 - val_accuracy: 0.0567
Epoch 5/50
22/22 [==============================] - 2s 111ms/step - loss: 0.3955 - accuracy: 0.8243 - val_loss: 1.6509 - val_accuracy: 0.2183
Epoch 6/50
22/22 [==============================] - 2s 111ms/step - loss: 0.2883 - accuracy: 0.8814 - val_loss: 2.5635 - val_accuracy: 0.2533
Epoch 7/50
22/22 [==============================] - 2s 112ms/step - loss: 0.2265 - accuracy: 0.9064 - val_loss: 2.3943 - val_accuracy: 0.3

In [ ]:
# 과대적합 보여짐
# 왜 이렇게 학습했을까?
# cnn 특성에 집중 → val 예측 성능은 떨어지는 상태
# 데이터는 복잡한 데이터일 수 있음
# 반대로 모델은 너무 단순할 수 있음

In [23]:
# 평가
# cnn_model.evaluate(X_test, y_test)[1] : 정확도만 확인
cnn_model.evaluate(X_test, y_test) # accuracy = 58%

32/32 [==============================] - 1s 20ms/step - loss: 4.8492 - accuracy: 0.5800


[4.84920597076416, 0.5799999833106995]